# Lecture 12: RNN1 - Basics
## Exercise 12-3: Combine RNN+Linear using embedding

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import sys

torch.manual_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach "hihell" -> "ihello"
x_data = [[0, 1, 0, 2, 3, 3]] # "hihell"
one_hot_lookup = [[[1, 0, 0, 0, 0],  # 0
                   [0, 1, 0, 0, 0],  # 1
                   [0, 0, 1, 0, 0],  # 2
                   [0, 0, 0, 1, 0],  # 3
                   [0, 0, 0, 0, 1]]] # 4

y_data = [1, 0, 2, 3, 3, 4] # "ihello"
x_one_hot = [[one_hot_lookup[0][x] for x in x_data[0]]]

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5      # one-hot size
hidden_size = 5     # ouput from the LSTM. 5 to directly predict one-hot
batch_size = 1      # one sentence
sequence_length = 6 # |ihello| == 6
num_layers = 1      # one-layer rnn

In [2]:
class CombinedModel(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(CombinedModel, self).__init__()

        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.input_size)
        self.embeddings = nn.Embedding(num_embeddings=self.input_size, 
                                       embedding_dim=self.hidden_size)
        
    def forward(self, x):
        emb = self.embeddings(x.long())

        hidden = self.init_hidden(x)
        
        # Reshape input in (batch_size, sequence_length, input_size)
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        x, hidden = self.rnn(x, hidden)
        out = self.linear(x)
        out = out.view(-1, num_classes)
        return out

    def init_hidden(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        return Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

In [3]:
# Instantiate RNN model
model = CombinedModel(num_classes, input_size, hidden_size, num_layers)
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

CombinedModel(
  (rnn): RNN(5, 5, batch_first=True)
  (linear): Linear(in_features=5, out_features=5, bias=True)
  (embeddings): Embedding(5, 5)
)


In [4]:
# Train the model
epochs = 100
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = "".join([idx2char[c] for c in idx.squeeze()])
    print(f"Epoch: {epoch}, Loss: {loss.data.item()}")
    print(f"Predicted string: {result_str}")

Epoch: 1, Loss: 1.6452089548110962
Predicted string: oooooo
Epoch: 2, Loss: 1.4475892782211304
Predicted string: llllll
Epoch: 3, Loss: 1.3083527088165283
Predicted string: llllll
Epoch: 4, Loss: 1.1435742378234863
Predicted string: illlll
Epoch: 5, Loss: 0.9536089897155762
Predicted string: ilello
Epoch: 6, Loss: 0.7698957324028015
Predicted string: ihello
Epoch: 7, Loss: 0.611672580242157
Predicted string: ihello
Epoch: 8, Loss: 0.48554909229278564
Predicted string: ihello
Epoch: 9, Loss: 0.3739349842071533
Predicted string: ihello
Epoch: 10, Loss: 0.2720254957675934
Predicted string: ihello
Epoch: 11, Loss: 0.1990143209695816
Predicted string: ihello
Epoch: 12, Loss: 0.14738725125789642
Predicted string: ihello
Epoch: 13, Loss: 0.10521908849477768
Predicted string: ihello
Epoch: 14, Loss: 0.07793185859918594
Predicted string: ihello
Epoch: 15, Loss: 0.05503419041633606
Predicted string: ihello
Epoch: 16, Loss: 0.04002940282225609
Predicted string: ihello
Epoch: 17, Loss: 0.031835149